In this notebook, I'm going to attempt to run all the techniques on the same circuit

In [89]:
import cirq
import numpy as np
from mitiq.benchmarks import generate_rb_circuits, ghz_circuits
from mitiq import MeasurementResult, Observable, PauliString, raw
from mitiq.shadows.shadows import *
from mitiq.shadows.shadows_utils import *

In [90]:
n_qubits = 2

In [91]:
circuit = generate_rb_circuits(2, 10)[0]

In [115]:
def execute(circuit: cirq.Circuit, noise_level: float = 0.002, p0: float = 0.05) -> MeasurementResult:
    """Execute a circuit with depolarizing noise of strength ``noise_level`` and readout errors ...
    """
    measurements = circuit[-1]
    circuit =  circuit[:-1]
    circuit = circuit.with_noise(cirq.depolarize(noise_level))
    circuit.append(cirq.bit_flip(p0).on_each(circuit.all_qubits()))
    circuit.append(measurements)

    simulator = cirq.DensityMatrixSimulator()

    result = simulator.run(circuit, repetitions=1000)
    bitstrings = np.column_stack(list(result.measurements.values()))
    return MeasurementResult(bitstrings)

In [93]:
obs = Observable(PauliString("ZI"), PauliString("IZ"))
noisy = raw.execute(circuit, execute, obs)

In [94]:
from functools import partial

ideal = raw.execute(circuit, partial(execute, noise_level=0, p0=0), obs)
print("Unmitigated value:", "{:.5f}".format(noisy.real))

Unmitigated value: 1.33800


In [95]:
from mitiq import rem

p0 = p1 = 0.05
icm = rem.generate_inverse_confusion_matrix(2, p0, p1)
rem_executor = rem.mitigate_executor(execute, inverse_confusion_matrix=icm)

rem_result = obs.expectation(circuit, rem_executor)
print("Mitigated value obtained with REM:", "{:.5f}".format(rem_result.real))

Mitigated value obtained with REM: 1.45400


In [9]:
from mitiq import zne

zne_executor = zne.mitigate_executor(execute, observable=obs, scale_noise=zne.scaling.folding.fold_global)
zne_result = zne_executor(circuit)
print("Mitigated value obtained with ZNE:", "{:.5f}".format(zne_result.real))

Mitigated value obtained with ZNE: -0.00000


In [38]:
from mitiq.pec import execute_with_pec
from mitiq.pec.representations.depolarizing import (
    represent_operations_in_circuit_with_local_depolarizing_noise
)
from mitiq import Executor

noisy_executor = Executor(execute)

noise_level = 0.01
reps = represent_operations_in_circuit_with_local_depolarizing_noise(circuit, noise_level)
print(f"{len(reps)} OperationRepresentation objects produced, assuming {100 * noise_level}% depolarizing noise.")

mitigated = execute_with_pec(
    circuit=circuit,
    executor=noisy_executor,
    representations=reps,
    observable=obs
)
mitigated_result = mitigated.real
print(f"Error-mitigated result with learning-based PEC: {mitigated_result:.5f}")

print(f"Error with mitigation (PEC): {abs(ideal - mitigated_result):.{3}}")

17 OperationRepresentation objects produced, assuming 1.0% depolarizing noise.


/Users/bigsad/Downloads/qrise-2024/.venv/lib/python3.11/site-packages/mitiq/pec/pec.py:123: LargeSampleWarning: The number of PEC samples is very large. It may take several minutes. It may be necessary to reduce 'precision' or 'num_samples'.
  warnings.warn(_LARGE_SAMPLE_WARN, LargeSampleWarning)


In [10]:
# classical shadows
shadow_quantum_processing(circuit, execute, 1)

Measurement:   0%|          | 0/1 [00:00<?, ?it/s]

(['10'], ['YX'])

In [11]:
# error rate of state reconstruction epsilon < 1.
epsilon = 1
# number of total measurements should perform for error rate epsilon
n_total_measurements = n_measurements_tomography_bound(epsilon, n_qubits)

print("n_total_measurements = {}".format(n_total_measurements))
shadow_outcomes = shadow_quantum_processing(
    circuit, execute, n_total_measurements
)

n_total_measurements = 544


In [20]:
# get shadow reconstruction of the density matrix
output = classical_post_processing(
    shadow_outcomes,
    observables=[PauliString("ZI"), PauliString("IZ")]
    # state_reconstruction=True,
)
# rho_shadow = output["reconstructed_state"]

In [21]:
output

{'Z(q(0))': 0.7610294117647058, 'Z(q(1))': 0.6783088235294118}

wtf why are there 2 values

In [12]:
combined_executor = zne.mitigate_executor(rem_executor, observable=obs, scale_noise=zne.scaling.folding.fold_global)

combined_result = combined_executor(circuit)
print("Mitigated value obtained with REM + ZNE:", "{:.5f}".format(combined_result.real))

Mitigated value obtained with REM + ZNE: 1.95920


In [26]:
# DDD
from mitiq import ddd

rule = ddd.rules.yy # low key I have no idea what this means

ddd_value = ddd.execute_with_ddd(
    circuit,
    execute,
    observable=obs,
    rule=rule,
)
print("Mitigated value obtained with DDD:", "{:.5f}".format(ddd_value.real))

Mitigated value obtained with DDD: 2.00000


In [29]:
# QSE
from mitiq import qse

Ms = [ # chat GPT generated stabilizer group
    "II",
    "XI",
    "ZI",
    "IX",
    "IZ"
]
check_operators = [
    PauliString(M, coeff=1, support=range(n_qubits)) for M in Ms
] # low key copy pasta-ed these so they may not even be the right choice
negative_Ms_as_pauliStrings = [
    PauliString(M, coeff=-1, support=range(n_qubits)) for M in Ms
] # low key copy pasta-ed these so they may not even be the right choice
code_hamiltonian = Observable(*negative_Ms_as_pauliStrings)

qse.execute_with_qse(
    circuit,
    execute,
    check_operators, # not sure what these are, need further reading
    code_hamiltonian,
    obs,
)

ValueError: Measuring an empty set of qubits.

In [126]:
# CDR
from mitiq import cdr

cdr_expval = cdr.execute_with_cdr(
    circuit,
    execute,
    observable=obs,
    simulator=partial(execute, noise_level=0, p0=0),
).real
print(f"mitigated expectation value with CDR: {cdr_expval:.2f}")

mitigated expectation value with CDR: 2.00


In [37]:
# # Pauli Twirling
# from mitiq import pt

# PT_circuit = pt.pauli_twirl_circuit(
#     circuit=circuit, # wait wtf but the noise isn't tied to the circuit???
# )
# # and wtf do I execute
# print(f"mitigated expectation value with Pauli Twirling: {PT_result:.2f}")

AttributeError: 'list' object has no attribute 'with_noise'

In [82]:
# DDD + ZNE
ddd_executor = ddd.mitigate_executor(execute, rule=rule)

ddd_zne_executor = zne.mitigate_executor(ddd_executor, observable=None, scale_noise=zne.scaling.folding.fold_global)

ddd_zne_result = ddd_zne_executor(circuit)
print("Mitigated value obtained with DDD + ZNE:", "{:.5f}".format(ddd_zne_result.real))

Mitigated value obtained with DDD + ZNE: 0.90398


In [85]:
# ZNE + DDD
zne_executor = zne.mitigate_executor(execute, scale_noise=zne.scaling
.folding.fold_global, observable=None)

zne_ddd_executor = ddd.mitigate_executor(zne_executor, observable=None, rule=rule)

zne_ddd_result = zne_ddd_executor(circuit)
print("Mitigated value obtained with DDD + ZNE:", "{:.5f}".format(zne_ddd_result.real))

Mitigated value obtained with DDD + ZNE: 0.90398


In [116]:
# REM + ZNE + DDD
rem_zne_executor = zne.mitigate_executor(rem_executor, scale_noise=zne.scaling
.folding.fold_global, observable=obs)

rem_zne_ddd_executor = ddd.mitigate_executor(rem_zne_executor, observable=obs, rule=rule)

rem_zne_ddd_result = rem_zne_ddd_executor(circuit)
print("Mitigated value obtained with REM + DDD + ZNE:", "{:.5f}".format(rem_zne_ddd_result.real))

ValueError: Cannot extract 2D measurements for repeated keys